In [21]:
from sympy.stats import Normal, density, cdf 
from sympy import Symbol, simplify, pprint, diff, log, sqrt, exp
from sympy.printing.numpy import NumPyPrinter, SciPyPrinter

## Standard Gaussian Density from sympy stats

In [2]:
x = Symbol("x")

N = Normal("N", 0, 1)

density(N)(x)

sqrt(2)*exp(-x**2/2)/(2*sqrt(pi))

## CDF

In [3]:
cdf(N)(x)

erf(sqrt(2)*x/2)/2 + 1/2

## Differentiating with respect to x returns the pdf

In [4]:
diff(cdf(N)(x), x)

sqrt(2)*exp(-x**2/2)/(2*sqrt(pi))

## Now define the PV of a European Call using the Black Model (lognormal forward diffusion)

In [22]:
# short rate
r = Symbol("r")
# forward
f = Symbol("f", positive=True)
# volatility
sigma = Symbol("sigma", positive=True)
# strike
k = Symbol("k", positive=True)
#time to maturity
t = Symbol("t", positive=True)

d1 = log(f/k)/(sigma*sqrt(t)) + 0.5*(sigma*sqrt(t))
d2 = d1 - sigma
pv = exp(-r*t) * (f*cdf(N)(d1) - k*cdf(N)(d2))

In [23]:
pv.simplify()

(f*(erf(sqrt(2)*(0.5*sigma**2*t + log(f/k))/(2*sigma*sqrt(t))) + 1) - k*(erf(sqrt(2)*(sigma**2*sqrt(t)*(0.5*sqrt(t) - 1) + log(f/k))/(2*sigma*sqrt(t))) + 1))*exp(-r*t)/2

## Delta Forward

In [25]:
delta_fwd = diff(pv, f).simplify()
delta_fwd

exp(-r*t)*erf(0.25*sqrt(2)*sigma*sqrt(t) + sqrt(2)*log(f/k)/(2*sigma*sqrt(t)))/2 + exp(-r*t)/2 + sqrt(2)*exp(-r*t - (0.5*sigma**2*t + log(f/k))**2/(2*sigma**2*t))/(2*sqrt(pi)*sigma*sqrt(t)) - sqrt(2)*k*exp(-r*t - (-sigma**2*sqrt(t) + 0.5*sigma**2*t + log(f/k))**2/(2*sigma**2*t))/(2*sqrt(pi)*f*sigma*sqrt(t))

## Theta

In [26]:
diff(pv, t).simplify()

(-2*pi*r*sigma*t**4*(f*(erf(sqrt(2)*(0.5*sigma**2*t + log(f/k))/(2*sigma*sqrt(t))) + 1) - k*(erf(sqrt(2)*(sigma**2*sqrt(t)*(0.5*sqrt(t) - 1) + log(f/k))/(2*sigma*sqrt(t))) + 1))*exp((2*r*sigma**2*t**2 + (0.5*sigma**2*t + log(f/k))**2 + (sigma**2*sqrt(t)*(0.5*sqrt(t) - 1) + log(f/k))**2)/(2*sigma**2*t)) + sqrt(2)*sqrt(pi)*t**(5/2)*(f*exp((sigma**2*sqrt(t)*(0.5*sqrt(t) - 1) + log(f/k))**2/(2*sigma**2*t)) - k*exp((0.5*sigma**2*t + log(f/k))**2/(2*sigma**2*t)))*(0.5*sigma**2*t - log(f/k))*exp(r*t))*exp(-(4*r*sigma**2*t**2 + (0.5*sigma**2*t + log(f/k))**2 + (sigma**2*sqrt(t)*(0.5*sqrt(t) - 1) + log(f/k))**2)/(2*sigma**2*t))/(4*pi*sigma*t**4)

## Vega

In [27]:
diff(pv, sigma).simplify()

sqrt(2)*(f*(0.5*sigma**2*t - log(f/k))*exp((sigma**2*sqrt(t)*(0.5*sqrt(t) - 1) + log(f/k))**2/(2*sigma**2*t)) + k*(sigma**2*sqrt(t)*(1 - 0.5*sqrt(t)) + log(f/k))*exp((0.5*sigma**2*t + log(f/k))**2/(2*sigma**2*t)))*exp(-(2*r*sigma**2*t**2 + (0.5*sigma**2*t + log(f/k))**2 + (sigma**2*sqrt(t)*(0.5*sqrt(t) - 1) + log(f/k))**2)/(2*sigma**2*t))/(2*sqrt(pi)*sigma**2*sqrt(t))

In [19]:
code = SciPyPrinter().doprint(delta_fwd)

print(code)

(1/2)*df*scipy.special.erf(0.25*numpy.sqrt(2)*sigma*numpy.sqrt(t) + (1/2)*numpy.sqrt(2)*numpy.log(f/k)/(sigma*numpy.sqrt(t))) + (1/2)*df + (1/2)*numpy.sqrt(2)*df*numpy.exp(-1/2*(0.5*sigma**2*t + numpy.log(f/k))**2/(sigma**2*t))/(numpy.sqrt(scipy.constants.pi)*sigma*numpy.sqrt(t)) - 1/2*numpy.sqrt(2)*df*k*numpy.exp(-1/2*(-sigma**2*numpy.sqrt(t) + 0.5*sigma**2*t + numpy.log(f/k))**2/(sigma**2*t))/(numpy.sqrt(scipy.constants.pi)*f*sigma*numpy.sqrt(t))
